# Лабораторная работа №3 "Теория кодирования".
Выполнили студенты:
Степанов Матвей и Овчинников Егор



In [1024]:
import numpy as np

In [1025]:
# Функция для генерации матрицы X
def generate_X(k, n):
    rows = []
    total_columns = n - k

    # Генерация всех возможных двоичных строк длины total_columns
    for i in range(1 << total_columns):
      row = [int(bit) for bit in bin(i)[2:].zfill(total_columns)]

      if sum(row) >= 2 and row not in rows:
        rows.append(row)

      if len(rows) >= k:
        break

    return np.array(rows, dtype=int)

## 3.1 Написать функцию формирования порождающей и проверочной матриц кода Хэмминга (𝟐𝒓−𝟏,𝟐𝒓−𝒓−𝟏,𝟑) на основе параметра 𝒓, а также таблицы синдромов для всех однократных ошибок.

In [1026]:
# Функция для генерации порождающей и проверочной матриц
def generate_hamming_matrices(r, n, k, x):
    # Порождающая матрица G (k x n)
    G = np.hstack((np.eye(k, dtype=int), x))

    # Проверочная матрица H (n - k) x n
    I_n_k = np.eye(n - k, dtype=int)
    H = np.hstack((x.T, I_n_k))

    return G, H.T

In [1027]:
def r_init(r):
  n = 2**r - 1  # Длина кодового слова
  k = 2**r - r - 1  # Длина информационного слова

  X = generate_X(k, n)
  G, H = generate_hamming_matrices(r, n, k, X)

  return G, H, n, k

In [1028]:
G, H, n, k = r_init(3)
print("Порождающая матрица G:\n", G)
print("Проверочная матрица H:\n", H)

Порождающая матрица G:
 [[1 0 0 0 0 1 1]
 [0 1 0 0 1 0 1]
 [0 0 1 0 1 1 0]
 [0 0 0 1 1 1 1]]
Проверочная матрица H:
 [[0 1 1]
 [1 0 1]
 [1 1 0]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]


## 3.2. Провести исследование кода Хэмминга для одно-, двух- и трёхкратных ошибок для 𝒓=𝟐,𝟑,𝟒.

# **r = 2**

In [1029]:
G, H, n, k = r_init(2)
print("Порождающая матрица G:\n", G)
print("Проверочная матрица H:\n", H)

Порождающая матрица G:
 [[1 1 1]]
Проверочная матрица H:
 [[1 1]
 [1 0]
 [0 1]]


In [1030]:
# Создание таблицы синдромов
syndromes = {}
for i in range(n):
    E = np.zeros(n, dtype=int)
    E[i] = 1
    S = (E @ H) % 2
    syndromes[tuple(S)] = tuple(E)

In [1031]:
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G) % 2
print("Отправлено:", codeword)

# Внесение однократной ошибки в кодовое слово
error_position = np.random.randint(0, n)
codeword_with_error = codeword.copy()
codeword_with_error[error_position] ^= 1
print("В ходе передачи возникла ошибка\nПринято:", codeword_with_error)

# Вычисление синдрома
syndrome = (codeword_with_error @ H) % 2
print("Синдром:", syndrome)

# Исправление ошибки
if tuple(syndrome) in syndromes:
    error_vector = syndromes[tuple(syndrome)]
    print("В принятом сообщении есть ошибка\nОшибка:", error_vector)

    corrected_codeword = codeword_with_error.copy()
    corrected_codeword[np.where(np.array(error_vector) == 1)[0]] ^= 1

    # Проверка на отличие от отправленного
    if not np.array_equal(corrected_codeword, codeword):
        print("Исправленное слово отличается от отправленного.")
    else:
        print("Исправленное слово совпадает с отправленным.")
else:
    corrected_codeword = codeword_with_error
print("Исправленное сообщение:", corrected_codeword)

Исходное сообщение: [1]
Отправлено: [1 1 1]
В ходе передачи возникла ошибка
Принято: [1 1 0]
Синдром: [0 1]
В принятом сообщении есть ошибка
Ошибка: (0, 0, 1)
Исправленное слово совпадает с отправленным.
Исправленное сообщение: [1 1 1]


# **r = 3**

In [1032]:
G, H, n, k = r_init(3)
print("Порождающая матрица G:\n", G)
print("Проверочная матрица H:\n", H)

Порождающая матрица G:
 [[1 0 0 0 0 1 1]
 [0 1 0 0 1 0 1]
 [0 0 1 0 1 1 0]
 [0 0 0 1 1 1 1]]
Проверочная матрица H:
 [[0 1 1]
 [1 0 1]
 [1 1 0]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]


In [1033]:
def generate_syndrome_table(H, n):
    syndrome_table = {}

    # Однократные ошибки
    for i in range(n):
        e = np.zeros(n, dtype=int)
        e[i] = 1  # Ошибка в позиции i
        syndrome = np.dot(e, H) % 2
        syndrome_table[tuple(syndrome)] = f"Ошибка на позиции {i + 1}"

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1  # Ошибка в позициях i и j
            syndrome = np.dot(e, H) % 2
            syndrome_table[tuple(syndrome)] = f"Ошибка на позиции ({i + 1}, {j + 1})"

    return syndrome_table

syndrome_table = generate_syndrome_table(H, n)

In [1034]:
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G) % 2
print("Отправлено:", codeword)

# Внесение двукратной ошибки в кодовое слово
error_positions = np.random.choice(n, 2, replace=False)
error_codeword = codeword.copy()
error_codeword[error_positions] ^= 1  # внесение ошибок
print("В ходе передачи возникла ошибка\nПринято:", error_codeword)

# Вычисление синдрома
syndrome = (error_codeword @ H) % 2
print("Синдром:", syndrome)

# Попытка исправить ошибку
error_correction = syndrome_table.get(tuple(syndrome), None)

if error_correction:
    corrected_codeword = np.copy(error_codeword)

    for pos in error_positions:
        corrected_codeword[pos] ^= 1
    print(f"Исправленное кодовое слово: {corrected_codeword}")
else:
    print("Ошибка не исправлена.")

# Проверка на отличие от отправленного
if not np.array_equal(corrected_codeword, codeword):
    print("Исправленное слово отличается от отправленного.")
else:
    print("Исправленное слово совпадает с отправленным.")

Исходное сообщение: [1 1 1 0]
Отправлено: [1 1 1 0 0 0 0]
В ходе передачи возникла ошибка
Принято: [1 1 0 0 0 1 0]
Синдром: [1 0 0]
Исправленное кодовое слово: [1 1 1 0 0 0 0]
Исправленное слово совпадает с отправленным.


# **r = 4**

In [1035]:
G, H, n, k = r_init(4)
print("Порождающая матрица G:\n", G)
print("Проверочная матрица H:\n", H)

Порождающая матрица G:
 [[1 0 0 0 0 0 0 0 0 0 0 0 0 1 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 1 1 0 1]
 [0 0 0 0 0 0 0 0 0 1 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]]
Проверочная матрица H:
 [[0 0 1 1]
 [0 1 0 1]
 [0 1 1 0]
 [0 1 1 1]
 [1 0 0 1]
 [1 0 1 0]
 [1 0 1 1]
 [1 1 0 0]
 [1 1 0 1]
 [1 1 1 0]
 [1 1 1 1]
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]]


In [1036]:
def generate_syndrome_table(H, n):
    syndrome_table = {}

    # Однократные ошибки
    for i in range(n):
        e = np.zeros(n, dtype=int)
        e[i] = 1  # Ошибка в позиции i
        syndrome = np.dot(e, H) % 2
        syndrome_table[tuple(syndrome)] = f"Ошибка на позиции {i + 1}"

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1  # Ошибка в позициях i и j
            syndrome = np.dot(e, H) % 2
            syndrome_table[tuple(syndrome)] = f"Ошибка на позиции ({i + 1}, {j + 1})"

    # Трехкратные ошибки
    for error_pos1 in range(n):
        for error_pos2 in range(error_pos1 + 1, n):
            for error_pos3 in range(error_pos2 + 1, n):
                error_vector = np.zeros(n, dtype=int)
                error_vector[error_pos1] = 1
                error_vector[error_pos2] = 1
                error_vector[error_pos3] = 1
                syndrome = (error_vector @ H) % 2
                syndrome_str = ''.join(map(str, syndrome))
                if syndrome_str in syndrome_table:
                    syndrome_table[syndrome_str].append((error_pos1 + 1, error_pos2 + 1, error_pos3 + 1))
                else:
                    syndrome_table[syndrome_str] = [(error_pos1 + 1, error_pos2 + 1, error_pos3 + 1)]

    return syndrome_table

syndrome_table = generate_syndrome_table(H, n)

In [1037]:
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G) % 2
print("Отправлено:", codeword)

# Внесение трехкратной ошибки в кодовое слово
error_positions = np.random.choice(n, 3, replace=False)
error_codeword = codeword.copy()
error_codeword[error_positions] ^= 1
print("В ходе передачи возникла ошибка\nПринято:", error_codeword)

# Вычисление синдрома
syndrome = (error_codeword @ H) % 2
print("Синдром:", syndrome)

# Попытка исправить ошибку
error_correction = syndrome_table.get(tuple(syndrome), None)

if error_correction:
    corrected_codeword = np.copy(error_codeword)

    for pos in error_positions:
        corrected_codeword[pos] ^= 1
    print(f"Исправленное кодовое слово: {corrected_codeword}")
else:
    print("Ошибка не исправлена.")

# Проверка на отличие от отправленного
if not np.array_equal(corrected_codeword, codeword):
    print("Исправленное слово отличается от отправленного.")
else:
    print("Исправленное слово совпадает с отправленным.")

Исходное сообщение: [1 0 1 0 1 1 1 0 1 0 1]
Отправлено: [1 0 1 0 1 1 1 0 1 0 1 1 1 1 1]
В ходе передачи возникла ошибка
Принято: [1 0 1 0 0 0 1 0 1 0 0 1 1 1 1]
Синдром: [1 1 0 0]
Исправленное кодовое слово: [1 0 1 0 1 1 1 0 1 0 1 1 1 1 1]
Исправленное слово совпадает с отправленным.


## 3.3 Написать функцию формирования порождающей и проверочной матриц расширенного кода Хэмминга (𝟐𝒓,𝟐𝒓−𝒓−𝟏,𝟑) на основе параметра 𝒓, а также таблицы синдромов для всех однократных ошибок.

In [1038]:
def generate_extended_hamming_matrices(r):
    G, H, n, k = r_init(r)

    # Расширение матрицы G
    parity_bits = np.mod(G.sum(axis=1), 2).astype(int)
    G_extended = np.hstack((G, parity_bits.reshape(-1, 1)))

    # Расширение матрицы H
    H_extended = np.hstack((H, np.ones((H.shape[0], 1), dtype=int)))
    overall_parity = np.mod(H_extended.sum(axis=0), 2).astype(int).reshape(1, -1)
    H_extended = np.vstack((H_extended, overall_parity))

    return G_extended, H_extended, n+1, k

In [1039]:
r = 3
G1, H1, n, k = generate_extended_hamming_matrices(r)
print("Порождающая матрица G:\n", G1)
print("Проверочная матрица H:\n", H1)

Порождающая матрица G:
 [[1 0 0 0 0 1 1 1]
 [0 1 0 0 1 0 1 1]
 [0 0 1 0 1 1 0 1]
 [0 0 0 1 1 1 1 0]]
Проверочная матрица H:
 [[0 1 1 1]
 [1 0 1 1]
 [1 1 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [0 1 0 1]
 [0 0 1 1]
 [0 0 0 1]]


## 3.4. Провести исследование расширенного кода Хэмминга для одно-, двух-, трёх- и четырёхкратных ошибок для 𝒓=𝟐,𝟑,𝟒.

# **r = 2**

In [1040]:
r = 2
G1, H1, n, k = generate_extended_hamming_matrices(r)
print("Порождающая матрица G:\n", G1)
print("Проверочная матрица H:\n", H1)

Порождающая матрица G:
 [[1 1 1 1]]
Проверочная матрица H:
 [[1 1 1]
 [1 0 1]
 [0 1 1]
 [0 0 1]]


In [1041]:
syndromes = {}
for i in range(n):
    E = np.zeros(n, dtype=int)
    E[i] = 1
    S = (E @ H1) % 2
    syndromes[tuple(S)] = tuple(E)

In [1042]:
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G1) % 2
print("Отправлено:", codeword)

# Внесение однократной ошибки в кодовое слово
error_position = np.random.randint(0, n)
codeword_with_error = codeword.copy()
codeword_with_error[error_position] ^= 1
print("В ходе передачи возникла ошибка\nПринято:", codeword_with_error)

# Вычисление синдрома
syndrome = (codeword_with_error @ H1) % 2
print("Синдром:", syndrome)

# Исправление ошибки
if tuple(syndrome) in syndromes:
    error_vector = syndromes[tuple(syndrome)]
    print("В принятом сообщении есть ошибка\nОшибка:", error_vector)

    corrected_codeword = codeword_with_error.copy()
    corrected_codeword[np.where(np.array(error_vector) == 1)[0]] ^= 1

    # Проверка на отличие от отправленного
    if not np.array_equal(corrected_codeword, codeword):
        print("Исправленное слово отличается от отправленного.")
    else:
        print("Исправленное слово совпадает с отправленным.")
else:
    corrected_codeword = codeword_with_error
print("Исправленное сообщение:", corrected_codeword)

Исходное сообщение: [1]
Отправлено: [1 1 1 1]
В ходе передачи возникла ошибка
Принято: [0 1 1 1]
Синдром: [1 1 1]
В принятом сообщении есть ошибка
Ошибка: (1, 0, 0, 0)
Исправленное слово совпадает с отправленным.
Исправленное сообщение: [1 1 1 1]


# **r = 3**

In [1043]:
r = 3
G1, H1, n, k = generate_extended_hamming_matrices(r)
print("Порождающая матрица G:\n", G1)
print("Проверочная матрица H:\n", H1)

Порождающая матрица G:
 [[1 0 0 0 0 1 1 1]
 [0 1 0 0 1 0 1 1]
 [0 0 1 0 1 1 0 1]
 [0 0 0 1 1 1 1 0]]
Проверочная матрица H:
 [[0 1 1 1]
 [1 0 1 1]
 [1 1 0 1]
 [1 1 1 1]
 [1 0 0 1]
 [0 1 0 1]
 [0 0 1 1]
 [0 0 0 1]]


In [1044]:
def generate_syndromes(H1):
    syndromes = {}

    # Однократные ошибки
    for i in range(H1.shape[0]):
        e = np.zeros(H1.shape[0], dtype=int)
        e[i] = 1
        syndrome = np.dot(e, H1) % 2
        syndromes[tuple(syndrome)] = f"Ошибка на позиции {i + 1}"

    # Двукратные ошибки
    for i in range(H1.shape[0]):
        for j in range(i + 1, H1.shape[0]):
            e = np.zeros(H1.shape[0], dtype=int)
            e[i] = 1
            e[j] = 1
            syndrome = np.dot(e, H1) % 2
            syndromes[tuple(syndrome)] = f"Ошибка на позиции ({i + 1}, {j + 1})"

    return syndromes

syndromes = generate_syndromes(H1)

In [1045]:
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G1) % 2
print("Отправлено:", codeword)

# Внесение двукратной ошибки в кодовое слово
error_positions = np.random.choice(n, 2, replace=False)
error_codeword = codeword.copy()
error_codeword[error_positions] ^= 1
print("В ходе передачи возникла ошибка\nПринято:", error_codeword)

# Вычисление синдрома
syndrome = (error_codeword @ H1) % 2
print("Синдром:", syndrome)

# Генерация таблицы синдромов
syndromes = generate_syndromes(H1)

# Попытка исправить ошибку
error_correction = syndromes.get(tuple(syndrome), None)

if error_correction:
    corrected_codeword = np.copy(error_codeword)

    for pos in error_positions:
        corrected_codeword[pos] ^= 1
    print(f"Исправленное кодовое слово: {corrected_codeword}")
else:
    print("Ошибка не исправлена.")

# Проверка на отличие от отправленного
if not np.array_equal(corrected_codeword, codeword):
    print("Исправленное слово отличается от отправленного.")
else:
    print("Исправленное слово совпадает с отправленным.")

Исходное сообщение: [1 0 0 0]
Отправлено: [1 0 0 0 0 1 1 1]
В ходе передачи возникла ошибка
Принято: [1 0 0 1 1 1 1 1]
Синдром: [0 1 1 0]
Исправленное кодовое слово: [1 0 0 0 0 1 1 1]
Исправленное слово совпадает с отправленным.


# **r = 4**

In [1046]:
r = 4
G1, H1, n, k = generate_extended_hamming_matrices(r)
print("Порождающая матрица G:\n", G1)
print("Проверочная матрица H:\n", H1)

Порождающая матрица G:
 [[1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1]
 [0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1]]
Проверочная матрица H:
 [[0 0 1 1 1]
 [0 1 0 1 1]
 [0 1 1 0 1]
 [0 1 1 1 1]
 [1 0 0 1 1]
 [1 0 1 0 1]
 [1 0 1 1 1]
 [1 1 0 0 1]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 1]
 [1 0 0 0 1]
 [0 1 0 0 1]
 [0 0 1 0 1]
 [0 0 0 1 1]
 [0 0 0 0 1]]


In [1047]:
def generate_syndromes(H1):
    syndromes = {}
    n = H1.shape[0]

    # Однократные ошибки
    for i in range(n):
        e = np.zeros(n, dtype=int)
        e[i] = 1  # Ошибка в позиции i
        syndrome = np.dot(e, H1) % 2
        syndromes[tuple(syndrome)] = f"Ошибка на позиции {i + 1}"

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1  # Ошибка в позициях i и j
            syndrome = np.dot(e, H1) % 2
            syndromes[tuple(syndrome)] = f"Ошибка на позиции ({i + 1}, {j + 1})"

    # Трехкратные ошибки
    for error_pos1 in range(n):
        for error_pos2 in range(error_pos1 + 1, n):
            for error_pos3 in range(error_pos2 + 1, n):
                error_vector = np.zeros(n, dtype=int)
                error_vector[error_pos1] = 1
                error_vector[error_pos2] = 1
                error_vector[error_pos3] = 1
                syndrome = (error_vector @ H1) % 2
                syndrome_str = ''.join(map(str, syndrome))
                if syndrome_str in syndromes:
                    syndromes[syndrome_str].append((error_pos1 + 1, error_pos2 + 1, error_pos3 + 1))
                else:
                    syndromes[syndrome_str] = [(error_pos1 + 1, error_pos2 + 1, error_pos3 + 1)]

    return syndromes

syndromes = generate_syndromes(H1)

In [1048]:
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G1) % 2
print("Отправлено:", codeword)

# Внесение трехкратной ошибки в кодовое слово
error_positions = np.random.choice(n, 3, replace=False)
error_codeword = codeword.copy()
error_codeword[error_positions] ^= 1
print("В ходе передачи возникла ошибка\nПринято:", error_codeword)

# Вычисление синдрома
syndrome = (error_codeword @ H1) % 2
print("Синдром:", syndrome)

# Генерация таблицы синдромов
syndromes = generate_syndromes(H1)

# Попытка исправить ошибку
error_correction = syndromes.get(tuple(syndrome), None)

if error_correction:
    corrected_codeword = np.copy(error_codeword)

    for pos in error_positions:
        corrected_codeword[pos] ^= 1
    print(f"Исправленное кодовое слово: {corrected_codeword}")
else:
    print("Ошибка не исправлена.")

# Проверка на отличие от отправленного
if not np.array_equal(corrected_codeword, codeword):
    print("Исправленное слово отличается от отправленного.")
else:
    print("Исправленное слово совпадает с отправленным.")

Исходное сообщение: [1 0 0 0 0 1 1 0 1 0 1]
Отправлено: [1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1]
В ходе передачи возникла ошибка
Принято: [1 0 0 0 0 1 1 0 1 1 1 0 1 1 0 1]
Синдром: [1 0 0 0 1]
Исправленное кодовое слово: [1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1]
Исправленное слово совпадает с отправленным.


# **r = 4 (четырехкратная ошибка)**

In [1049]:
def generate_syndromes(H1):
    syndromes = {}
    n = H1.shape[0]

    # Однократные ошибки
    for i in range(n):
        e = np.zeros(n, dtype=int)
        e[i] = 1  # Ошибка в позиции i
        syndrome = np.dot(e, H1) % 2
        syndromes[tuple(syndrome)] = f"Ошибка на позиции {i + 1}"

    # Двукратные ошибки
    for i in range(n):
        for j in range(i + 1, n):
            e = np.zeros(n, dtype=int)
            e[i] = 1
            e[j] = 1  # Ошибка в позициях i и j
            syndrome = np.dot(e, H1) % 2
            syndromes[tuple(syndrome)] = f"Ошибка на позиции ({i + 1}, {j + 1})"

    # Трехкратные ошибки
    for error_pos1 in range(n):
        for error_pos2 in range(error_pos1 + 1, n):
            for error_pos3 in range(error_pos2 + 1, n):
                error_vector = np.zeros(n, dtype=int)
                error_vector[error_pos1] = 1
                error_vector[error_pos2] = 1
                error_vector[error_pos3] = 1
                syndrome = (error_vector @ H1) % 2
                syndrome_str = ''.join(map(str, syndrome))
                if syndrome_str in syndromes:
                    syndromes[syndrome_str].append((error_pos1 + 1, error_pos2 + 1, error_pos3 + 1))
                else:
                    syndromes[syndrome_str] = [(error_pos1 + 1, error_pos2 + 1, error_pos3 + 1)]

    # Четырехкратные ошибки
    for error_pos1 in range(n):
        for error_pos2 in range(error_pos1 + 1, n):
            for error_pos3 in range(error_pos2 + 1, n):
                for error_pos4 in range(error_pos3 + 1, n):
                    error_vector = np.zeros(n, dtype=int)
                    error_vector[error_pos1] = 1
                    error_vector[error_pos2] = 1
                    error_vector[error_pos3] = 1
                    error_vector[error_pos4] = 1
                    syndrome = (error_vector @ H1) % 2
                    syndrome_str = ''.join(map(str, syndrome))
                    if syndrome_str in syndromes:
                        syndromes[syndrome_str].append((error_pos1 + 1, error_pos2 + 1, error_pos3 + 1, error_pos4 + 1))
                    else:
                        syndromes[syndrome_str] = [(error_pos1 + 1, error_pos2 + 1, error_pos3 + 1, error_pos4 + 1)]

    return syndromes

syndromes = generate_syndromes(H1)

In [1050]:
message = np.random.randint(0, 2, size=k)
print("Исходное сообщение:", message)
codeword = (message @ G1) % 2
print("Отправлено:", codeword)

# Внесение четырехкратной ошибки в кодовое слово
error_positions = np.random.choice(n, 4, replace=False)
error_codeword = codeword.copy()
error_codeword[error_positions] ^= 1
print("В ходе передачи возникла ошибка\nПринято:", error_codeword)

# Вычисление синдрома
syndrome = (error_codeword @ H1) % 2
print("Синдром:", syndrome)

# Генерация таблицы синдромов
syndromes = generate_syndromes(H1)

# Попытка исправить ошибку
error_correction = syndromes.get(tuple(syndrome), None)

if error_correction:
    corrected_codeword = np.copy(error_codeword)

    for pos in error_positions:
        corrected_codeword[pos] ^= 1
    print(f"Исправленное кодовое слово: {corrected_codeword}")
else:
    print("Ошибка не исправлена.")

# Проверка на отличие от отправленного
if not np.array_equal(corrected_codeword, codeword):
    print("Исправленное слово отличается от отправленного.")
else:
    print("Исправленное слово совпадает с отправленным.")

Исходное сообщение: [1 1 0 1 0 0 1 0 0 0 0]
Отправлено: [1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0]
В ходе передачи возникла ошибка
Принято: [1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0]
Синдром: [0 0 0 1 0]
Исправленное кодовое слово: [1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0]
Исправленное слово совпадает с отправленным.
